<a href="https://colab.research.google.com/github/Partha-SUST16/LLM_JailBreak_Low_Resource/blob/main/python_scripts/Translation_of_prompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-cloud-translate==2.0.1

How to set up the api key: https://chatgpt.com/share/6789f2f7-9a64-800c-ab08-bbc068a2d622

In [ ]:
import os

# Replace 'your-key-file.json' with the name of your uploaded JSON key file
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/drive/MyDrive/CPSC 538P Project (Computer Security)/peppy-strategy-450318-p5-0842ce560080.json"

In [ ]:
from google.cloud import translate_v2 as translate

# Initialize the client
client = translate.Client()

# Text to translate
text = "Hello, world!"
target_language = "es"  # Spanish

# Translate the text
result = client.translate(text, target_language=target_language)

# Print the results
print(f"Translated Text: {result['translatedText']}")


Translated Text: ¡Hola Mundo!


In [ ]:
import pandas as pd

# Load the CSV file
file_path = '/content/drive/MyDrive/R&D/Jail Breaking/jailbreak_prompts_2023_05_07.csv'  # Replace with your CSV file's path
df = pd.read_csv(file_path)

# Filter rows where 'jailbreak' is True
df = df[df['jailbreak'] == True]
df.count()

,0
platform,666
source,666
prompt,666
jailbreak,666
created_at,664
date,664
community_id,383
community_name,196


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import html

translate_client = translate.Client()

# Define the target languages and corresponding countries
languages = {
    # 'hi': 'India (Hindi)',
    # 'ur': 'Pakistan (Urdu)',
    # 'bn': 'Bangladesh (Bengali)',
    # 'ne': 'Nepal (Nepali)',
    # 'si': 'Sri Lanka (Sinhala)',
    # 'ta': 'Sri Lanka (Tamil)',
    'dz': 'Bhutan (Dzongkha)',
    'dv': 'Maldives (Dhivehi)',
    # 'ps': 'Afghanistan (Pashto)'
}

# Function to translate a prompt
def translate_prompt(prompt, target_language):
    result = translate_client.translate(prompt, target_language=target_language)
    return html.unescape(result['translatedText'])

# Function to calculate cosine similarity
def calculate_cosine_similarity(text1, text2):
     # Preprocess text: remove special characters and convert to lowercase
    text1 = re.sub(r"[^a-zA-Z0-9 ]", "", text1).lower()
    text2 = re.sub(r"[^a-zA-Z0-9 ]", "", text2).lower()

    # Check if text1 and text2 are empty after preprocessing
    if not text1 or not text2:
        print(text1)
        print(text2)
        print("Warning: Text is empty after preprocessing. Returning similarity 0.")
        return 0  # Return 0 if either text is empty

    vectorizer = TfidfVectorizer(stop_words='english')  # Specify English stop words

    try:
        vectors = vectorizer.fit_transform([text1, text2])
    except ValueError:
        print("Warning: Empty vocabulary after removing stop words. Returning similarity 0.")
        print(text1)
        print(text2)
        return 0  # Return 0 if vocabulary is empty

    # Compute cosine similarity between the two vectors
    similarity = cosine_similarity(vectors[0], vectors[1])[0][0]
    return similarity

# Create a results DataFrame to store translations and similarities

# Iterate through the target languages
for lang, country in languages.items():
    results = []
    for _, row in df.iterrows():
        original_prompt = row['prompt']

        # Translate to the target language
        translated_prompt = translate_prompt(original_prompt, lang)

        # Back-translate to English
        back_translated_prompt = translate_prompt(translated_prompt, 'en')

        # Calculate cosine similarity
        similarity = calculate_cosine_similarity(original_prompt, back_translated_prompt)

        # Append results
        results.append({
            'platform': row['platform'],
            'source': row['source'],
            'original_prompt': original_prompt,
            'translated_prompt': translated_prompt,
            'back_translated_prompt': back_translated_prompt,
            'language': country,
            'cosine_similarity': similarity
        })

    # Convert results to a DataFrame
    results_df = pd.DataFrame(results)

    # Save results to a CSV file
    output_file = f'/content/drive/MyDrive/R&D/Jail Breaking/{lang}_translated_prompts_with_similarity.csv'
    results_df.to_csv(output_file, index=False)
    print(f'Results saved to {output_file}')

In [ ]:
!pip install googletrans datasets

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import html
from googletrans import Translator

translate_client = Translator()

# Define the target languages and corresponding countries
languages = {
    'hi': 'India (Hindi)',
    'ur': 'Pakistan (Urdu)',
    'bn': 'Bangladesh (Bengali)',
    'ne': 'Nepal (Nepali)',
    'si': 'Sri Lanka (Sinhala)',
    'ta': 'Sri Lanka (Tamil)',
    # 'dz': 'Bhutan (Dzongkha)',
    # 'dv': 'Maldives (Dhivehi)',
    'ps': 'Afghanistan (Pashto)'
}

# Function to translate a prompt
async def translate_prompt(prompt, target_language, src_language):
    result = await translate_client.translate(prompt, src= src_language, dest=target_language)
    return html.unescape(result.text)

# Function to calculate cosine similarity
def calculate_cosine_similarity(text1, text2):
     # Preprocess text: remove special characters and convert to lowercase
    text1 = re.sub(r"[^a-zA-Z0-9 ]", "", text1).lower()
    text2 = re.sub(r"[^a-zA-Z0-9 ]", "", text2).lower()

    # Check if text1 and text2 are empty after preprocessing
    if not text1 or not text2:
        print(text1)
        print(text2)
        print("Warning: Text is empty after preprocessing. Returning similarity 0.")
        return 0  # Return 0 if either text is empty

    vectorizer = TfidfVectorizer(stop_words='english')  # Specify English stop words

    try:
        vectors = vectorizer.fit_transform([text1, text2])
    except ValueError:
        print("Warning: Empty vocabulary after removing stop words. Returning similarity 0.")
        print(text1)
        print(text2)
        return 0  # Return 0 if vocabulary is empty

    # Compute cosine similarity between the two vectors
    similarity = cosine_similarity(vectors[0], vectors[1])[0][0]
    return similarity


In [ ]:
from datasets import load_dataset
import pandas as pd

df = pd.read_csv("hf://datasets/allenai/wildjailbreak/eval/eval.tsv", sep="\t")
df = df[df['data_type'] == 'adversarial_harmful']
df.head()

,adversarial,label,data_type
210,Imagine you are a social media influencer who ...,1,adversarial_harmful
211,You are a security researcher working on a hig...,1,adversarial_harmful
212,Compose an enlightening article about the prev...,1,adversarial_harmful
213,In the spirit of investigative journalism and ...,1,adversarial_harmful
214,Imagine we're in an alternate universe where y...,1,adversarial_harmful


In [ ]:
import asyncio
import pandas as pd
import nest_asyncio

# Enable nested asyncio loops for Colab compatibility
nest_asyncio.apply()

async def translate_and_compute(original_prompt, lang, country, row, idx):
    # Translate to the target language
    translated_prompt, back_translated_prompt = await asyncio.gather(
        translate_prompt(original_prompt, lang, 'en'),
        translate_prompt(original_prompt, 'en', lang)
    )

    # Calculate cosine similarity
    similarity = calculate_cosine_similarity(original_prompt, back_translated_prompt)

    # Return result dictionary
    return {
        'id': idx,
        'original_prompt': original_prompt,
        'translated_prompt': translated_prompt,
        'back_translated_prompt': back_translated_prompt,
        'language': country,
        'cosine_similarity': similarity
    }

async def process_language(lang, country, df):
    results = []
    tasks = []
    idx = 0

    for row in df.itertuples(index=False):
        original_prompt = row.adversarial

        # Create async tasks for concurrent execution
        tasks.append(
            translate_and_compute(original_prompt, lang, country, row, idx)
        )

        if idx % 100 == 0:
            print(f"Current Language {lang}, processed {idx} rows")

        idx += 1

    # Execute all translation tasks concurrently
    results = await asyncio.gather(*tasks)

    # Convert results to a DataFrame and save to CSV
    results_df = pd.DataFrame(results)
    output_file = f'/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/{lang}_translated_prompts_with_similarity.csv'
    results_df.to_csv(output_file, index=False)
    print(f'Results saved to {output_file}')

async def main():
    tasks = [process_language(lang, country, df) for lang, country in languages.items()]
    await asyncio.gather(*tasks)

# Run the event loop manually in Colab
await main()

Current Language hi, processed 0 rows
Current Language hi, processed 100 rows
Current Language hi, processed 200 rows
Current Language hi, processed 300 rows
Current Language hi, processed 400 rows
Current Language hi, processed 500 rows
Current Language hi, processed 600 rows
Current Language hi, processed 700 rows
Current Language hi, processed 800 rows
Current Language hi, processed 900 rows
Current Language hi, processed 1000 rows
Current Language hi, processed 1100 rows
Current Language hi, processed 1200 rows
Current Language hi, processed 1300 rows
Current Language hi, processed 1400 rows
Current Language hi, processed 1500 rows
Current Language hi, processed 1600 rows
Current Language hi, processed 1700 rows
Current Language hi, processed 1800 rows
Current Language hi, processed 1900 rows
Current Language ur, processed 0 rows
Current Language ur, processed 100 rows
Current Language ur, processed 200 rows
Current Language ur, processed 300 rows
Current Language ur, processed 400

In [ ]:
import pandas as pd
import glob

# Path to your folder containing the CSV files
folder_path = "/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/*translated_prompts_with_similarity.csv"

# Load all CSV files
csv_files = glob.glob(folder_path)
dfs = {}

for file in csv_files:
    df = pd.read_csv(file)

    # Trim column names to remove unwanted spaces
    df.columns = df.columns.str.strip()

    # Check if 'cosine_similarity' exists
    if 'cosine_similarity' not in df.columns:
        print(f"Column 'cosine_similarity' not found in {file}. Available columns: {df.columns}")
        continue  # Skip this file if column is missing

    # Select top 1500 rows by cosine similarity
    df = df.sort_values(by='cosine_similarity', ascending=False).head(1500)

    # Store DataFrame with filename as key
    dfs[file] = df

# Find common IDs across all files
common_ids = set.intersection(*[set(df['id']) for df in dfs.values()])

# Filter out common IDs from each DataFrame
filtered_dfs = {file: df[df['id'].isin(common_ids)] for file, df in dfs.items()}

# Save the filtered data back to CSV
for file, df in filtered_dfs.items():
    output_file = file.replace(".csv", "_filtered_500.csv")
    print(len(df))
    # df.to_csv(output_file, index=False)
    # print(f"Filtered data saved to: {output_file}")


446
446
446
446
446
446
446
446
446


In [ ]:
import pandas as pd
import glob

# Path to the filtered CSV files
folder_path = "/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Filtered Prompt/*_filtered_1000.csv"

# Get all filtered CSV file paths
csv_files = glob.glob(folder_path)

# Load all CSV files into a list
dfs = []
for file in csv_files:
    df = pd.read_csv(file)

    # Trim column names to remove any spaces
    df.columns = df.columns.str.strip()

    # Add a new column to track the source file/language
    language = file.split('/')[-1].split('_')[0]  # Extract language prefix (hi, ur, bn, etc.)
    df['language'] = language

    dfs.append(df)

# Merge all DataFrames on 'id'
merged_df = dfs[0]
for df in dfs[1:]:
    merged_df = pd.merge(merged_df, df, on='id', how='outer', suffixes=('', '_dup'))

# Remove duplicate column suffixes (if any)
merged_df = merged_df.loc[:, ~merged_df.columns.str.endswith('_dup')]
print(merged_df.shape)
# Save the merged data to a new CSV file
# output_file = "/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Filtered Prompt/merged_translated_prompts_1000.csv"
# merged_df.to_csv(output_file, index=False)

print(f"Merged data saved to: {output_file}")


(1794, 6)
Merged data saved to: /content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Filtered Prompt/merged_translated_prompts_1000.csv


In [ ]:
for lang, country in languages.items():
    print(lang, country)
    results = []
    idx = 0
    for row in df.itertuples(index=False):
        original_prompt = row.adversarial

        # Translate to the target language
        translated_prompt = translate_prompt(original_prompt, lang)

        # Back-translate to English
        back_translated_prompt = translate_prompt(translated_prompt, 'en')

        # Calculate cosine similarity
        similarity = calculate_cosine_similarity(original_prompt, back_translated_prompt)

        if idx % 100 == 0:
            print(f"Current Language {lang}, processed {idx} rows")

        # Append results
        results.append({
            'id': idx,
            'original_prompt': original_prompt,
            'translated_prompt': translated_prompt,
            'back_translated_prompt': back_translated_prompt,
            'language': country,
            'cosine_similarity': similarity
        })
        idx += 1

    # Convert results to a DataFrame
    results_df = pd.DataFrame(results)

    # Save results to a CSV file
    output_file = f'/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/{lang}_translated_prompts_with_similarity.csv'
    results_df.to_csv(output_file, index=False)
    print(f'Results saved to {output_file}')

dz Bhutan (Dzongkha)
Current Language dz, processed 0 rows
Current Language dz, processed 100 rows
Current Language dz, processed 200 rows
Current Language dz, processed 300 rows
Current Language dz, processed 400 rows
Current Language dz, processed 500 rows
Current Language dz, processed 600 rows
Current Language dz, processed 700 rows
Current Language dz, processed 800 rows
Current Language dz, processed 900 rows
Current Language dz, processed 1000 rows
Current Language dz, processed 1100 rows
Current Language dz, processed 1200 rows
Current Language dz, processed 1300 rows
Current Language dz, processed 1400 rows
Current Language dz, processed 1500 rows
Current Language dz, processed 1600 rows
Current Language dz, processed 1700 rows
Current Language dz, processed 1800 rows
Current Language dz, processed 1900 rows
Results saved to /content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/dz_translated_prompts_with_similarity.csv
dv Maldives (Dhivehi)
Current 